In [1]:
from google.colab import files
uploaded = files.upload()

Saving 2000-2012_Export.csv to 2000-2012_Export.csv
Saving 2000-2012_Import.csv to 2000-2012_Import.csv
Saving 2013-2024_Export.csv to 2013-2024_Export.csv
Saving 2013-2024_Import.csv to 2013-2024_Import.csv
Saving Core_economic_indicators.csv to Core_economic_indicators.csv
Saving crop_and_livestock.csv to crop_and_livestock.csv
Saving disasters.csv to disasters.csv
Saving Employment_Unemployment.csv to Employment_Unemployment.csv
Saving population_and_demographics.csv to population_and_demographics.csv
Saving Resiliance.csv to Resiliance.csv
Saving Social_and_welfare.csv to Social_and_welfare.csv


In [57]:
import pandas as pd
# Load each dataset individually
gdp = pd.read_csv("Core_economic_indicators.csv", encoding="latin1")
trade_export_1 = pd.read_csv("2000-2012_Export.csv", encoding="latin1")
trade_import_1 = pd.read_csv("2000-2012_Import.csv", encoding="latin1")
trade_export_2 = pd.read_csv("2013-2024_Export.csv", encoding="latin1")
trade_import_2 = pd.read_csv("2013-2024_Import.csv", encoding="latin1")
crops = pd.read_csv("crop_and_livestock.csv", encoding="latin1")
disasters = pd.read_csv("disasters.csv", encoding="latin1")
employment = pd.read_csv("Employment_Unemployment.csv", encoding="latin1")
pop = pd.read_csv("population_and_demographics.csv", encoding="latin1")
resilience = pd.read_csv("Resiliance.csv", encoding="latin1")
welfare = pd.read_csv("Social_and_welfare.csv", encoding="latin1")

In [58]:
#Standardize column names
if "Area" in pop.columns:
    pop.rename(columns={"Area":"Country"}, inplace=True)
if "Country Name" in employment.columns:
    employment.rename(columns={"Country Name":"Country"}, inplace=True)

In [59]:
#reshape the columns (clean Year column (from "2000 [YR2000]" -> 2000))
def reshape_wide(df, value_name):
    df_long = df.melt(
        id_vars=["Country Name", "Series Name"],
        var_name="Year",
        value_name=value_name
    )
    df_long["Year"] = df_long["Year"].str.extract(r"(\d{4})").astype("Int64") #avoids NaN-to-int error
    df_long.rename(columns={"Country Name": "Country", "Series Name": "Indicator"}, inplace=True)
    return df_long

gdp_long = reshape_wide(gdp, "GDP_Value")
welfare_long = reshape_wide(welfare, "Welfare_Value")
resilience_long = reshape_wide(resilience, "Resilience_Value")

#clean employment
employment_long = employment.melt(
    id_vars=["Country", "Series Name"],
    var_name="Year",
    value_name="Employment_Value"
)
employment_long["Year"] = employment_long["Year"].str.extract(r"(\d{4})").astype("Int64")
employment_long.rename(columns={"Series Name":"Indicator"}, inplace=True)

In [60]:
#renaming column value to population_value
pop_renamed = pop.rename(columns={"Value": "Population_Value"})
print(pop_renamed.head())

for df in [gdp_long, welfare_long, resilience_long, employment_long, pop_renamed, trade_all, crop_pivot, disaster_summary]:
    if "Country Name" in df.columns:
        df.rename(columns={"Country Name": "Country"}, inplace=True)
    if "Area" in df.columns:
        df.rename(columns={"Area": "Country"}, inplace=True)
    if "reporterDesc" in df.columns:
        df.rename(columns={"reporterDesc": "Country"}, inplace=True)

# 2. Make sure all datasets have numeric "Year"
for df in [gdp_long, welfare_long, resilience_long, employment_long, pop_renamed, trade_all, crop_pivot, disaster_summary]:
    if "Year" in df.columns:
        df["Year"] = pd.to_numeric(df["Year"], errors="coerce")

  ï»¿Domain Code             Domain  Area Code (M49)      Country  \
0             OA  Annual population                4  Afghanistan   
1             OA  Annual population                4  Afghanistan   
2             OA  Annual population                4  Afghanistan   
3             OA  Annual population                4  Afghanistan   
4             OA  Annual population                4  Afghanistan   

   Element Code                        Element  Item Code  \
0           511  Total Population - Both sexes       3010   
1           512        Total Population - Male       3010   
2           513      Total Population - Female       3010   
3           551               Rural population       3010   
4           561               Urban population       3010   

                        Item  Year Code  Year     Unit  Population_Value Flag  \
0  Population - Est. & Proj.       2000  2000  1000 No         20130.327    X   
1  Population - Est. & Proj.       2000  2000  1000 No  

In [62]:
#merge dataset into one (cleaned_dataset)

# --- Layer 1: Core socio-economic indicators (all have "Indicator")
core = gdp_long.copy()
core = pd.merge(core, welfare_long, on=["Country","Year","Indicator"], how="outer")
core = pd.merge(core, resilience_long, on=["Country","Year","Indicator"], how="outer")
core = pd.merge(core, employment_long, on=["Country","Year","Indicator"], how="outer")

# --- Layer 2: Merge population (no Indicator)
cleaned_dataset = pd.merge(core,
                           pop_renamed[["Country","Year","Population_Value"]],
                           on=["Country","Year"], how="left")

# --- Layer 3: Merge Trade
cleaned_dataset = pd.merge(cleaned_dataset, trade_all,
                           on=["Country","Year"], how="left")

# --- Layer 4: Merge Crops
cleaned_dataset = pd.merge(cleaned_dataset, crop_pivot,
                           on=["Country","Year"], how="left")

# --- Layer 5: Merge Disasters
cleaned_dataset = pd.merge(cleaned_dataset, disaster_summary,
                           on=["Country","Year"], how="left")

print("✅ After merging all datasets:", cleaned_dataset.shape)
print(cleaned_dataset.head())

✅ After merging all datasets: (103785, 20)
       Country                           Indicator  Year GDP_Value  \
0  Afghanistan  Current account balance (% of GDP)  2000       NaN   
1  Afghanistan  Current account balance (% of GDP)  2000       NaN   
2  Afghanistan  Current account balance (% of GDP)  2000       NaN   
3  Afghanistan  Current account balance (% of GDP)  2000       NaN   
4  Afghanistan  Current account balance (% of GDP)  2000       NaN   

  Welfare_Value Resilience_Value Employment_Value  Population_Value  \
0           NaN               ..              NaN         20130.327   
1           NaN               ..              NaN         10094.646   
2           NaN               ..              NaN         10035.682   
3           NaN               ..              NaN         15657.474   
4           NaN               ..              NaN          4436.282   

   primaryValue  cifvalue  Chickens     Maize      Beef  Potatoes   Cow_Milk  \
0           NaN       NaN    

In [63]:
#prepare & merge Trade Data (Exports + Imports)
#function to clean and aggregate trade data
def prepare_trade(df, value_col="primaryValue"):
    if "reporterDesc" in df.columns and "refYear" in df.columns:
        return df.groupby(["reporterDesc","refYear"])[value_col].sum().reset_index().rename(
            columns={"reporterDesc":"Country","refYear":"Year", value_col:value_col}
        )
    return pd.DataFrame()
# Process exports and imports separately
exp1 = prepare_trade(trade_export_1)             # Exports 2000–2012
imp1 = prepare_trade(trade_import_1, "cifvalue") # Imports 2000–2012
exp2 = prepare_trade(trade_export_2)             # Exports 2013–2024
imp2 = prepare_trade(trade_import_2, "cifvalue") # Imports 2013–2024

# combine all years together
trade_all = pd.concat([exp1,exp2], ignore_index=True)
imp_all = pd.concat([imp1,imp2], ignore_index=True)

# merge Exports + Imports into one table
trade_all = pd.merge(trade_all, imp_all, on=["Country","Year"], how="outer")

# merge into cleaned dataset
cleaned_dataset = pd.merge(cleaned_dataset, trade_all, on=["Country","Year"], how="left")

In [64]:
# prepare & Merge Crop & Livestock Data
# Rename Area -> Country for consistency
crops.rename(columns={"Area": "Country"}, inplace=True)

# shortlist of key items
important_items = [
    "Wheat",
    "Rice",
    "Maize (corn)",
    "Potatoes",
    "Sugar cane",
    "Raw milk of cattle",
    "Meat of cattle with the bone, fresh or chilled",
    "Chickens"
]

crop_filtered = crops[crops["Item"].isin(important_items)]
# pivot : each Item becomes its own column
crop_pivot = crop_filtered.pivot_table(
    index=["Country", "Year"],
    columns="Item",
    values="Value",
    aggfunc="sum"
).reset_index()
# renaming columns to simpler names
crop_pivot.rename(columns={
    "Maize (corn)": "Maize",
    "Raw milk of cattle": "Cow_Milk",
    "Meat of cattle with the bone, fresh or chilled": "Beef",
    "Sugar cane": "Sugarcane"
}, inplace=True)

# merge into cleaned dataset
cleaned_dataset = pd.merge(cleaned_dataset, crop_pivot, on=["Country", "Year"], how="left")

In [65]:
#prepare & Merge Disaster Data
#summarize disaster impacts by Country + Year
if "Total Deaths" in disasters.columns:
    disaster_summary = disasters.groupby(["Country", "Start Year"]).agg({
        "Total Deaths": "sum",
        "Total Damage ('000 US$)": "sum"
    }).reset_index()

    # renaming Start Year -> Year and clean column names
    disaster_summary.rename(columns={
        "Start Year": "Year",
        "Total Deaths": "Disaster_Deaths",
        "Total Damage ('000 US$)": "Disaster_Damages"
    }, inplace=True)

    # merge into clean dataset
    cleaned_dataset = pd.merge(cleaned_dataset, disaster_summary, on=["Country","Year"], how="left")

In [67]:
#merge dataset into one (cleaned_dataset)

# --- Layer 1: Core socio-economic indicators (all have "Indicator")
core = gdp_long.copy()
core = pd.merge(core, welfare_long, on=["Country","Year","Indicator"], how="outer")
core = pd.merge(core, resilience_long, on=["Country","Year","Indicator"], how="outer")
core = pd.merge(core, employment_long, on=["Country","Year","Indicator"], how="outer")

# --- Layer 2: Merge population (no Indicator)
cleaned_dataset = pd.merge(core,
                           pop_renamed[["Country","Year","Population_Value"]],
                           on=["Country","Year"], how="left")

# --- Layer 3: Merge Trade
cleaned_dataset = pd.merge(cleaned_dataset, trade_all,
                           on=["Country","Year"], how="left")

# --- Layer 4: Merge Crops
cleaned_dataset = pd.merge(cleaned_dataset, crop_pivot,
                           on=["Country","Year"], how="left")

# --- Layer 5: Merge Disasters
cleaned_dataset = pd.merge(cleaned_dataset, disaster_summary,
                           on=["Country","Year"], how="left")

print("After merging all datasets:", cleaned_dataset.shape)
print(cleaned_dataset.head())

After merging all datasets: (103785, 20)
       Country                           Indicator  Year GDP_Value  \
0  Afghanistan  Current account balance (% of GDP)  2000       NaN   
1  Afghanistan  Current account balance (% of GDP)  2000       NaN   
2  Afghanistan  Current account balance (% of GDP)  2000       NaN   
3  Afghanistan  Current account balance (% of GDP)  2000       NaN   
4  Afghanistan  Current account balance (% of GDP)  2000       NaN   

  Welfare_Value Resilience_Value Employment_Value  Population_Value  \
0           NaN               ..              NaN         20130.327   
1           NaN               ..              NaN         10094.646   
2           NaN               ..              NaN         10035.682   
3           NaN               ..              NaN         15657.474   
4           NaN               ..              NaN          4436.282   

   primaryValue  cifvalue  Chickens     Maize      Beef  Potatoes   Cow_Milk  \
0           NaN       NaN    68

In [68]:
#clean merged dataset
#fill missing values forward
cleaned_dataset.ffill(inplace=True)
#also backward fill if some values remain missing
cleaned_dataset.bfill(inplace=True)
# remove duplicates
cleaned_dataset.drop_duplicates(inplace=True)
# infer correct data types (numbers, dates, etc.)
cleaned_dataset = cleaned_dataset.infer_objects(copy=False)

#ensure numeric columns are properly converted
#converting all object (string) columns that should be numeric
for col in cleaned_dataset.columns:
    if cleaned_dataset[col].dtype == "object":
        cleaned_dataset[col] = pd.to_numeric(cleaned_dataset[col], errors="coerce")

print("Numeric conversion done")
print(cleaned_dataset.dtypes.head(20))
print("Cleaning done. Shape after cleaning:", cleaned_dataset.shape)

Numeric conversion done
Country             float64
Indicator           float64
Year                  Int64
GDP_Value           float64
Welfare_Value       float64
Resilience_Value    float64
Employment_Value    float64
Population_Value    float64
primaryValue        float64
cifvalue            float64
Chickens            float64
Maize               float64
Beef                float64
Potatoes            float64
Cow_Milk            float64
Rice                float64
Sugarcane           float64
Wheat               float64
Disaster_Deaths     float64
Disaster_Damages    float64
dtype: object
Cleaning done. Shape after cleaning: (100456, 20)


In [69]:
#feature engineering
if "primaryValue" in cleaned_dataset.columns and "cifvalue" in cleaned_dataset.columns and "GDP_Value" in cleaned_dataset.columns:
    cleaned_dataset["Trade_Dependency"] = (cleaned_dataset["primaryValue"] + cleaned_dataset["cifvalue"]) / (cleaned_dataset["GDP_Value"]+1)

if "GDP_Value" in cleaned_dataset.columns and "Welfare_Value" in cleaned_dataset.columns:
    cleaned_dataset["Resilience_Score"] = (cleaned_dataset["GDP_Value"] - cleaned_dataset["Welfare_Value"]) / 10

if "Population_Value" in cleaned_dataset.columns and "Employment_Value" in cleaned_dataset.columns:
    cleaned_dataset["Population_Pressure"] = cleaned_dataset["Population_Value"] * cleaned_dataset["Employment_Value"]

In [70]:
#saving it as a csv file
cleaned_dataset.to_csv("final_dataset.csv", index=False)
#files.download("final_dataset.csv")

print("Final dataset ready! Shape:", cleaned_dataset.shape)
print(cleaned_dataset.head())

Final dataset ready! Shape: (100456, 23)
   Country  Indicator  Year  GDP_Value  Welfare_Value  Resilience_Value  \
0      NaN        NaN  2000        NaN            NaN               NaN   
1      NaN        NaN  2000        NaN            NaN               NaN   
2      NaN        NaN  2000        NaN            NaN               NaN   
3      NaN        NaN  2000        NaN            NaN               NaN   
4      NaN        NaN  2000        NaN            NaN               NaN   

   Employment_Value  Population_Value  primaryValue  cifvalue  ...  Potatoes  \
0             12.82         20130.327           NaN       NaN  ...  265785.7   
1             12.82         10094.646           NaN       NaN  ...  265785.7   
2             12.82         10035.682           NaN       NaN  ...  265785.7   
3             12.82         15657.474           NaN       NaN  ...  265785.7   
4             12.82          4436.282           NaN       NaN  ...  265785.7   

    Cow_Milk      Rice  Sug